In [298]:
import numpy as np
import pandas as pd 
np.set_printoptions(suppress=True, precision=3)
pd.set_option('precision', 3)

In [299]:
def regression(X, b, Y):
    if isinstance(X, pd.DataFrame): X = X.values 
    if isinstance(Y, pd.DataFrame) or isinstance(Y, pd.Series): Y = Y.values 

    for x, y in zip(X, Y):
        print('y = ' + ' + '.join([f'{b_.round(2)}*{x_.round(2)}' for (x_, b_) in zip(x, b)]) + f' = {y.round(3)}')   

1) Запишемо рівняння регресії:

In [300]:
'ŷ=b0 + b1x1 + b2x2 + b3x3 + b12x1x2 + b13x1x3 + b23x2x3 + b123x1x2x3'

'ŷ=b0 + b1x1 + b2x2 + b3x3 + b12x1x2 + b13x1x3 + b23x2x3 + b123x1x2x3'

2) Матриця планування експерименту.

In [301]:
Xenc = np.array([
    # first side
    [1, -1, -1, -1],
    [1, -1, +1, +1],
    [1, +1, -1, +1],
    [1, +1, +1, -1],
    # second side
    [1, +1, +1, +1],
    [1, +1, -1, -1],
    [1, -1, +1, -1],
    [1, -1, -1, +1]
])
pd.DataFrame(Xenc, columns=['x0', 'x1', 'x2', 'x3'])

,x0,x1,x2,x3
0,1,-1,-1,-1
1,1,-1,1,1
2,1,1,-1,1
3,1,1,1,-1
4,1,1,1,1
5,1,1,-1,-1
6,1,-1,1,-1
7,1,-1,-1,1


3) Для розрахунку коефіцієнтів рівняння регресії для натуральних значень запишемо матрицю
планування з відповідними натуралізованими значеннями факторів.

In [302]:
x1, x2, x3 = (15, 45), (-25, 10), (45, 50) 

columns = []
for c1 in ['x1', 'x2', 'x3']: columns += [(c1, 'min'), (c1, 'max')]
    
df = pd.DataFrame(data=[[*x1, *x2, *x3]], columns=columns, index=['105'])
df.columns = pd.MultiIndex.from_tuples(columns)
df

x1      x2      x3    
    min max min max min max
105  15  45 -25  10  45  50

In [303]:
X = np.copy(Xenc[:, 1:])
xmap = [x1, x2, x3]
for i in range(X.shape[1]):
    X[:, i] = np.where(X[:, i] == -1, *xmap[i])


x_cp_max = sum(map(max, [x1, x2, x3])) / 3
x_cp_min = sum(map(min, [x1, x2, x3])) / 3
y_imax, y_imin = 200 + x_cp_max, 200 + x_cp_min

m = 3
Y = np.random.randint(y_imin, y_imax, size=(8, m))

MP = pd.DataFrame(np.concatenate([X, Y], axis=1), columns=['x1', 'x2', 'x3', 'y1', 'y2', 'y3'])
MP 

,x1,x2,x3,y1,y2,y3
0,15,-25,45,226,231,214
1,15,10,50,227,220,219
2,45,-25,50,227,217,219
3,45,10,45,216,224,224
4,45,10,50,229,233,227
5,45,-25,45,231,229,221
6,15,10,45,226,224,229
7,15,-25,50,219,212,214


Sklearn пакет для машинного навчання, там є все починаючи з методів пониження розмірності(PCA, tSNE), стандартизацій даних і закінчуючи простого перцептрону або якогось SVM

In [304]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(3, interaction_only=True)
poly_x = poly.fit_transform(X)

MP = pd.concat([pd.DataFrame(poly_x, columns=['x0', 'x1', 'x2', 'x3', 'x1x2', 'x1x3', 'x2x3', 'x1x2x3']), MP.iloc[:, -3:]], axis=1)
MP 

,x0,x1,x2,x3,x1x2,x1x3,x2x3,x1x2x3,y1,y2,y3
0,1.0,15.0,-25.0,45.0,-375.0,675.0,-1125.0,-16875.0,226,231,214
1,1.0,15.0,10.0,50.0,150.0,750.0,500.0,7500.0,227,220,219
2,1.0,45.0,-25.0,50.0,-1125.0,2250.0,-1250.0,-56250.0,227,217,219
3,1.0,45.0,10.0,45.0,450.0,2025.0,450.0,20250.0,216,224,224
4,1.0,45.0,10.0,50.0,450.0,2250.0,500.0,22500.0,229,233,227
5,1.0,45.0,-25.0,45.0,-1125.0,2025.0,-1125.0,-50625.0,231,229,221
6,1.0,15.0,10.0,45.0,150.0,675.0,450.0,6750.0,226,224,229
7,1.0,15.0,-25.0,50.0,-375.0,750.0,-1250.0,-18750.0,219,212,214


In [305]:
MP['ym'] = MP.iloc[:, -3:].mean(axis=1)
MP

,x0,x1,x2,x3,x1x2,x1x3,x2x3,x1x2x3,y1,y2,y3,ym
0,1.0,15.0,-25.0,45.0,-375.0,675.0,-1125.0,-16875.0,226,231,214,223.667
1,1.0,15.0,10.0,50.0,150.0,750.0,500.0,7500.0,227,220,219,222.000
2,1.0,45.0,-25.0,50.0,-1125.0,2250.0,-1250.0,-56250.0,227,217,219,221.000
3,1.0,45.0,10.0,45.0,450.0,2025.0,450.0,20250.0,216,224,224,221.333
4,1.0,45.0,10.0,50.0,450.0,2250.0,500.0,22500.0,229,233,227,229.667
5,1.0,45.0,-25.0,45.0,-1125.0,2025.0,-1125.0,-50625.0,231,229,221,227.000
6,1.0,15.0,10.0,45.0,150.0,675.0,450.0,6750.0,226,224,229,226.333
7,1.0,15.0,-25.0,50.0,-375.0,750.0,-1250.0,-18750.0,219,212,214,215.000


In [306]:
MP.iloc[:, :-4].mean(axis=0) # mx

x0            1.00
x1           30.00
x2           -7.50
x3           47.50
x1x2       -225.00
x1x3       1425.00
x2x3       -356.25
x1x2x3   -10687.50
dtype: float64

In [307]:
MP['ym'].mean() # my

223.25

In [308]:
X = MP.iloc[:, :-4].copy().values
 
A = np.array([(X * X[:, i]).sum(axis=0) for i in range(X.shape[1])])
y = np.array([(MP['ym'] * X[:, i]).sum() for i in range(X.shape[1])])
b = np.linalg.solve(A.T, y)
b

array([169.815,   1.716,  -2.598,   1.089,   0.043,  -0.036,   0.054,
        -0.001])

In [310]:
# Ax = y -> x коэффициенты 

regression(MP.iloc[:, :-4], b, MP['ym'])

y = 169.82*1.0 + 1.72*15.0 + -2.6*-25.0 + 1.09*45.0 + 0.04*-375.0 + -0.04*675.0 + 0.05*-1125.0 + -0.0*-16875.0 = 223.667
y = 169.82*1.0 + 1.72*15.0 + -2.6*10.0 + 1.09*50.0 + 0.04*150.0 + -0.04*750.0 + 0.05*500.0 + -0.0*7500.0 = 222.0
y = 169.82*1.0 + 1.72*45.0 + -2.6*-25.0 + 1.09*50.0 + 0.04*-1125.0 + -0.04*2250.0 + 0.05*-1250.0 + -0.0*-56250.0 = 221.0
y = 169.82*1.0 + 1.72*45.0 + -2.6*10.0 + 1.09*45.0 + 0.04*450.0 + -0.04*2025.0 + 0.05*450.0 + -0.0*20250.0 = 221.333
y = 169.82*1.0 + 1.72*45.0 + -2.6*10.0 + 1.09*50.0 + 0.04*450.0 + -0.04*2250.0 + 0.05*500.0 + -0.0*22500.0 = 229.667
y = 169.82*1.0 + 1.72*45.0 + -2.6*-25.0 + 1.09*45.0 + 0.04*-1125.0 + -0.04*2025.0 + 0.05*-1125.0 + -0.0*-50625.0 = 227.0
y = 169.82*1.0 + 1.72*15.0 + -2.6*10.0 + 1.09*45.0 + 0.04*150.0 + -0.04*675.0 + 0.05*450.0 + -0.0*6750.0 = 226.333
y = 169.82*1.0 + 1.72*15.0 + -2.6*-25.0 + 1.09*50.0 + 0.04*-375.0 + -0.04*750.0 + 0.05*-1250.0 + -0.0*-18750.0 = 215.0


**Урахуванням взаємодії для нормованих значень факторів.**
1. Сума усіх нормованих значень стовпчика кожного фактора дорівнює нулю (симетричність)
2. Сума попарних добутків стовпчиків факторів дорівнює нулю (ортогональність)
3. Сума добутків трьох стовпчиків факторів також дорівнює нулю
4. Оскільки квадрат нормованих значень ±1 завжди дорівнює 1, то сума квадратів дорівнює N

In [311]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(3, interaction_only=True)
poly_x = poly.fit_transform(Xenc[:, 1:])

MPenc = pd.DataFrame(poly_x, columns=['x0', 'x1', 'x2', 'x3', 'x1x2', 'x1x3', 'x2x3', 'x1x2x3'])
MPenc

,x0,x1,x2,x3,x1x2,x1x3,x2x3,x1x2x3
0,1.0,-1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0
1,1.0,-1.0,1.0,1.0,-1.0,-1.0,1.0,-1.0
2,1.0,1.0,-1.0,1.0,-1.0,1.0,-1.0,-1.0
3,1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5,1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0
6,1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,1.0
7,1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,1.0


In [312]:
# MPenc.sum(axis=0)

In [313]:
# poly = PolynomialFeatures(3, interaction_only=True, include_bias=False)
# poly.fit_transform(MPenc.iloc[:, 1:]).sum(axis=0)

In [314]:
N = 8
A = np.eye(N, N) * N
A

array([[8., 0., 0., 0., 0., 0., 0., 0.],
       [0., 8., 0., 0., 0., 0., 0., 0.],
       [0., 0., 8., 0., 0., 0., 0., 0.],
       [0., 0., 0., 8., 0., 0., 0., 0.],
       [0., 0., 0., 0., 8., 0., 0., 0.],
       [0., 0., 0., 0., 0., 8., 0., 0.],
       [0., 0., 0., 0., 0., 0., 8., 0.],
       [0., 0., 0., 0., 0., 0., 0., 8.]])

In [315]:
X = MPenc.copy().values
y = np.array([(MP['ym'] * X[:, i]).sum() for i in range(X.shape[1])])

# Ax = y -> x коэффициенты 
b = np.linalg.solve(A, y)
b

array([223.25 ,   1.5  ,   1.583,  -1.333,  -0.833,   1.917,   2.333,
         1.25 ])

In [316]:
regression(MPenc, b, MP['ym'])

y = 223.25*1.0 + 1.5*-1.0 + 1.58*-1.0 + -1.33*-1.0 + -0.83*1.0 + 1.92*1.0 + 2.33*1.0 + 1.25*-1.0 = 223.667
y = 223.25*1.0 + 1.5*-1.0 + 1.58*1.0 + -1.33*1.0 + -0.83*-1.0 + 1.92*-1.0 + 2.33*1.0 + 1.25*-1.0 = 222.0
y = 223.25*1.0 + 1.5*1.0 + 1.58*-1.0 + -1.33*1.0 + -0.83*-1.0 + 1.92*1.0 + 2.33*-1.0 + 1.25*-1.0 = 221.0
y = 223.25*1.0 + 1.5*1.0 + 1.58*1.0 + -1.33*-1.0 + -0.83*1.0 + 1.92*-1.0 + 2.33*-1.0 + 1.25*-1.0 = 221.333
y = 223.25*1.0 + 1.5*1.0 + 1.58*1.0 + -1.33*1.0 + -0.83*1.0 + 1.92*1.0 + 2.33*1.0 + 1.25*1.0 = 229.667
y = 223.25*1.0 + 1.5*1.0 + 1.58*-1.0 + -1.33*-1.0 + -0.83*-1.0 + 1.92*-1.0 + 2.33*1.0 + 1.25*1.0 = 227.0
y = 223.25*1.0 + 1.5*-1.0 + 1.58*1.0 + -1.33*-1.0 + -0.83*-1.0 + 1.92*1.0 + 2.33*-1.0 + 1.25*1.0 = 226.333
y = 223.25*1.0 + 1.5*-1.0 + 1.58*-1.0 + -1.33*1.0 + -0.83*1.0 + 1.92*-1.0 + 2.33*-1.0 + 1.25*1.0 = 215.0


4. Провести 3 статистичні перевірки – за критеріями Кохрена, Стьюдента, Фішера.

**Критерій Кохрена**

In [321]:
from statsmodels.stats.contingency_tables import cochrans_q
print(cochrans_q(MP[['y1', 'y2', 'y3']]))

df          2
pvalue      0.36787944117144245
statistic   2.0


In [365]:
def cochran_q(X, p=0.05, return_Gt=False):
    """ Перевірка однорідності дисперсії за критерієм Кохрена """
    uniform = True
    m = X.shape[1] - 1 # f1
    N = X.shape[0] # f2
    
    Xvar = ((X - X.mean(axis=1, keepdims=True)) ** 2).sum(axis=1) / m # Xvar = X.var(axis=1)
    Gt = 0.5157 # таблица
    G = Xvar.max() / Xvar.sum()
    
    if G > Gt: uniform = False
    return (G, Gt, uniform) if return_Gt else (G, uniform)

    
G, Gt, uniform = cochran_q(MP[['y1', 'y2', 'y3']].values, return_Gt=True) # m = 2, N = 8 
if uniform: 
    print(f'G < Gt, {G:.3f} < {Gt:.3f} - Дисперсія однорідна.')
else: 
    print(f'G > Gt, {G:.3f} > {Gt:.3f} - Дисперсія неоднорідна.')

G < Gt, 0.379 < 0.516 - Дисперсія однорідна.


**Критерій Стьюдента:**
* Перший спосіб це оцінка p_value яка визначається як scipy.stats.t.sf(t, df), t - список t_значень, а df - степені свободи 
* Другий спосіб як в методичці, результат виходить той самий 

In [401]:
from scipy.stats import t as T

def t_test(X, Y, p=0.05):
    """ Значимість коефіцієнтів регресії згідно критерію Стьюдента
        :return: nd.array - індекси значимих коефіцієнтів рівняння регресії
    """
    N, m = Y.shape
    df = (m - 1) * N # f3 = f1 * f2 

    var = Y.var(axis=1).mean()
    S2 = var / (m * N)    
    betas = (X.T * Y.mean(axis=1)).mean(axis=1)
    t = abs(betas) / np.sqrt(S2)
    
    ind = np.argwhere(T.sf(t, df) * 2 > p).ravel()
    return t, betas, ind

               
t, betas, ind = t_test(MPenc.values, MP[['y1', 'y2', 'y3']].values)

print(f'{" ".join([f"t{i} = {t[i]:.3f}" for i in ind])} < Tтабл.')
print(f'{" ".join([f"b{i}" for i in ind])} коефіцієнти рівняння регресії приймаємо незначними при рівні значимості 0.05\n') 

t1 = 1.794 t2 = 1.894 t3 = 1.595 t4 = 0.997 t7 = 1.495 < Tтабл.
b1 b2 b3 b4 b7 коефіцієнти рівняння регресії приймаємо незначними при рівні значимості 0.05



In [435]:
def t_test(X, Y, p=0.05):
    """ 
    Значимість коефіцієнтів регресії згідно критерію Стьюдента
    :return: nd.array - індекси значимих коефіцієнтів рівняння регресії
    """
    N, m = Y.shape
    df = (m - 1) * N # f3 = f1 * f2 
    tt = 2.12  # таблиця


    var = Y.var(axis=1).mean()
    S2 = var / (m * N)    
    betas = (X.T * Y.mean(axis=1)).mean(axis=1)
    t = abs(betas) / np.sqrt(S2)
    
    ind_out = np.argwhere(t < tt).ravel()
    ind_in = np.argwhere(t > tt).ravel()
    return t, betas, ind_out, ind_in, tt, m, N
    
t, betas, ind_out, ind_in, tt, m, N = t_test(MPenc.values, MP[['y1', 'y2', 'y3']].values)

print(f'{" ".join([f"t{i} = {t[i]:.3f}" for i in ind_out])} < Tтабл. = {tt}')
print(f'{" ".join([f"b{i}" for i in ind_out])} коефіцієнти рівняння регресії приймаємо незначними при рівні значимості 0.05\n')

t1 = 1.794 t2 = 1.894 t3 = 1.595 t4 = 0.997 t7 = 1.495 < Tтабл. = 2.12
b1 b2 b3 b4 b7 коефіцієнти рівняння регресії приймаємо незначними при рівні значимості 0.05



In [436]:
MPenc.iloc[:, ind_in] # значимі 

,x0,x1x3,x2x3
0,1.0,1.0,1.0
1,1.0,-1.0,1.0
2,1.0,1.0,-1.0
3,1.0,-1.0,-1.0
4,1.0,1.0,1.0
5,1.0,-1.0,1.0
6,1.0,1.0,-1.0
7,1.0,-1.0,-1.0


In [437]:
regression(MPenc.iloc[:, ind_in], b[ind_in], MPenc.iloc[:, ind_in] @ b[ind_in])

y = 223.25*1.0 + 1.92*1.0 + 2.33*1.0 = 227.5
y = 223.25*1.0 + 1.92*-1.0 + 2.33*1.0 = 223.667
y = 223.25*1.0 + 1.92*1.0 + 2.33*-1.0 = 222.833
y = 223.25*1.0 + 1.92*-1.0 + 2.33*-1.0 = 219.0
y = 223.25*1.0 + 1.92*1.0 + 2.33*1.0 = 227.5
y = 223.25*1.0 + 1.92*-1.0 + 2.33*1.0 = 223.667
y = 223.25*1.0 + 1.92*1.0 + 2.33*-1.0 = 222.833
y = 223.25*1.0 + 1.92*-1.0 + 2.33*-1.0 = 219.0


**Критерій Фішера**

In [470]:
from scipy.stats import f as F

def F_test(y1, y2, d, m, N):
    """ 
    Адекватність моделі за F-критерієм Фішера, який дорівнює відношенню дисперсії
    адекватності до дисперсії відтворюваності:
    """
    
    S2ad = (m / (N - d)) * ((y2 - y1.mean(axis=1)) ** 2).sum()
    S2 = Y.var(axis=1).mean()
    Fval = S2ad / S2 
    Ft = 2.9 # таблиця 
    
    p = F.sf(Fval, (m - 1) * N, N - d)
    print(p.round(3), p > 0.05)

    adequate = Fval < Ft
    return Fval, Ft, adequate

    
d = X.shape[1] - len(ind) # кількість значимих коефіцієнтів 
Fval, Ft, adequate = F_test(MP[['y1', 'y2', 'y3']].values, (MPenc.iloc[:, ind_in] @ b[ind_in]).values, d, m, N)

message = lambda x1, x2: f'Fp {x1} Ft, {Fval:.3f} {x1} {Ft:.3f}\nрівняння регресії {x2}адекватнe оригіналу при рівні значимості 0.05'
if not adequate:
    print(message('>', 'не'))
else:
    print(message('<', ''))

0.157 True
Fp < Ft, 2.515 < 2.900
рівняння регресії адекватнe оригіналу при рівні значимості 0.05
